In [14]:
import pandas as pd
from pathlib import Path
from datetime import datetime

In [15]:
df_path = Path(r"C:\Users\n740789\Documents\Projects_local\DataSets\DATAFEED\ficheros_tratados\2025\20250401_Equities_feed_IssuerLevel_sinOVR.csv")
crossreference_path = Path(r"C:\Users\n740789\Documents\clarity_data_quality_controls\excel_books\aladdin_data\crossreference\Aladdin_Clarity_Issuers_20250401.csv")

In [16]:
df = pd.read_csv(df_path, low_memory=False, encoding="utf-8", dtype={"permid":str})
crossreference = pd.read_csv(crossreference_path, low_memory=False, encoding="utf-8", dtype={"CLARITY_AI":str})    

In [17]:
crossreference.columns = crossreference.columns.str.strip().str.lower()

In [20]:
crossreference.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134991 entries, 0 to 134990
Data columns (total 5 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   aladdin_issuer  134991 non-null  object 
 1   issuer_name     134991 non-null  object 
 2   permid          131893 non-null  object 
 3   msci            65026 non-null   object 
 4   sust            23308 non-null   float64
dtypes: float64(1), object(4)
memory usage: 5.1+ MB


In [19]:
crossreference.rename(columns={"clarity_ai":"permid"}, inplace=True)

In [21]:
df = df.merge(crossreference[["aladdin_issuer", "permid"]], how="left", on='permid')

In [22]:
# reorder df columns so that "aladdin_issuer" is the first column and "permid" is the second column
cols = df.columns.tolist()
cols.insert(0, cols.pop(cols.index("aladdin_issuer")))
cols.insert(1, cols.pop(cols.index("permid")))
df = df[cols]

In [25]:
df.iloc[:, :10].info()

<class 'pandas.core.frame.DataFrame'>
Index: 51590 entries, 0 to 69327
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   aladdin_issuer      51590 non-null  object
 1   permid              51590 non-null  object
 2   isin                51590 non-null  object
 3   instrument_type     51590 non-null  object
 4   issuer_name         51590 non-null  object
 5   issuer_country      51029 non-null  object
 6   gics2_industry      43644 non-null  object
 7   region              51002 non-null  object
 8   company_inheriting  51590 non-null  bool  
 9   parent_company      51590 non-null  object
dtypes: bool(1), object(9)
memory usage: 4.0+ MB


In [24]:
# if aladdin_issuer nan drop row
df.dropna(subset=["aladdin_issuer"], inplace=True)

In [26]:
# add columnn datadate with date in string format
df["datadate"] = datetime.strptime(df_path.stem.split("_")[0], "%Y%m%d").strftime("%Y%m%d")

In [27]:
df.datadate.value_counts()

datadate
20250401    51590
Name: count, dtype: int64

In [28]:
df.to_csv(Path(r"C:\Users\n740789\Downloads\20250401_clarity_equities_issuer.csv"), index=False)

In [3]:
#df_path = Path(r"C:\Users\n740789\Documents\Projects_local\DataSets\DATAFEED\ficheros_tratados\2025\20250401_Equities_feed_IssuerLevel_sinOVR.csv")
#target_cols = ['permid','issuer_name', 'company_inheriting','gics2_industry', 'region','sustainability_rating']
#df = pd.read_csv(df_path, low_memory=False, usecols=target_cols)
#target_ratings = ['Sufficient', 'Limited', 'Poor']
## print number of empty gics2_industry
##print(df.gics2_industry.isna().sum())
## print number of empty region
##print(df.region.isna().sum())
## print number of empty gics & region
##print(df[(df.gics2_industry.isna()) & (df.region.isna())].shape[0])
## get new df with rows where gics2_industry, or region or both are empty
#empty_gics2_industry = df[df.gics2_industry.isna()]
## get final df where sustainability_rating is in target_ratings
#final_df = empty_gics2_industry[empty_gics2_industry.sustainability_rating.isin(target_ratings)]
## save final_df to csv on downloads
#final_df.to_csv(Path(r"C:\Users\n740789\Downloads\final_df.csv"), index=False)